<a href="https://colab.research.google.com/github/amanjain487/tsai-eva6/blob/main/Assignments/S8/Advanced%20Training%20Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%rm -rf 'CIFAR_10'
!git clone https://github.com/amanjain487/CIFAR_10

Cloning into 'CIFAR_10'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 117 (delta 57), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (117/117), 28.81 KiB | 7.20 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
import sys
sys.path.append('/CIFAR_10/')

!pip install git+https://github.com/albumentations-team/albumentations.git


  Cloning https://github.com/albumentations-team/albumentations.git to /tmp/pip-req-build-77jzme53
  Running command git clone -q https://github.com/albumentations-team/albumentations.git /tmp/pip-req-build-77jzme53
  Created wheel for albumentations: filename=albumentations-1.0.0-cp37-none-any.whl size=98151 sha256=2e79b0d32316c8b25d75d752a152af3604cf50bfaee769bbfafa293e6b171f92
  Stored in directory: /tmp/pip-ephem-wheel-cache-fy9p94ue/wheels/e2/85/3e/2a40fac5cc1f43ced656603bb2fca1327b30ec7de1b1b66517
Successfully built albumentations
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [3]:
from CIFAR_10.models import resnet

In [4]:
from CIFAR_10 import utils
from CIFAR_10 import main

ModuleNotFoundError: ignored

In [ ]:
device, best_acc, classes, net, criterion, optimizer, scheduler = main.define_model_utilities()

In [ ]:
tensor_transforms, numpy_transforms = utils.train_transform(["totensor", "normalize_mean", "randomcrop", "horizontal_flip", "cutout", "random_rotate"])

In [ ]:
train_set, test_set = utils.load_dataset(tensor_transforms, numpy_transforms)

In [ ]:
train_loader, test_loader = main.dataloaders(train_set, test_set)

In [ ]:
train_loss, train_acc, test_loss, test_acc = main.start_training(40, net, criterion, optimizer, device, train_loader, test_loader, best_acc, scheduler)